In [ ]:
import numpy as np
import scipy.stats
import astropy
import pylab as plt

In [ ]:

from astroML.datasets import fetch_dr7_quasar

# Fetch the quasar data
quasars = fetch_dr7_quasar()

# select the first 10000 points
quasars = quasars[:10000]

data = quasars['redshift']

In [ ]:
plt.hist(data,bins=50,histtype='step',density=True,label='original data');

A numpy histogram object out of the data with `density=True` and `bins=50` is a tuple of bin heights and bin edges.

In [ ]:
np.histogram(data, bins=100,density=True)

### Cloning by rejection sampling

In [ ]:
plt.hist(data,bins=50,histtype='step',density=True,label='original data');

# make a simple histogram object
counts, bins = np.histogram(data, bins=50, density=True)
maxh = counts.max() # find the maximum

# Make a scipy.stats random variable object from a histogram
# This is a great hack!
disth = scipy.stats.rv_histogram((counts,bins))

# Let's do it manually again
N = 100000 # trials
q = np.random.uniform(-10, 30, N) # proposed points
u = np.random.uniform(0, maxh, N) # uniform draws

mask = u<=disth.pdf(q) # assess whether u <= q(x_i)

monte_carlo = q[mask] # reject all points that don't pass, using masking

plt.hist(monte_carlo, bins=50, density=True,histtype='step',label='cloned data 1');

### But scipy has it already implemented 
plt.hist(disth.rvs(size=N),bins=50,density=True,histtype='step',label='cloned data 2');

plt.legend();

### Cloning by inverse transform

In [ ]:
plt.hist(data,bins=50,histtype='step',density=True,label='original data');


# make a simple histogram object
counts, bins = np.histogram(data, bins=50, density=True)
bin_mids = (bins[1:] + bins[:-1]) / 2 # mid location of bins
 
simple_cdf = np.cumsum(counts) / np.sum(counts) # very simply cumulative sum

# set up an interpolation of the inverse cumulative distribution
tck = scipy.interpolate.interp1d(simple_cdf, bin_mids)

# sample evenly along the cumulative distribution, and interpolate
# little hack to make sure no points are generated outside interpolation range.
# not ideal
u = np.random.uniform(min(simple_cdf),max(simple_cdf), 10000) 
x_sample = tck(u)

plt.hist(x_sample, bins=100, density=True, histtype='step',label='cloned data');

plt.legend();

## Now some cosmology...

Let's try to assume that quasars are distributed uniformly in comoving volume in the Universe. Seems fair...

We use the cosmological parameters as measured by the Plack satellite, which is a flat $\Lambda$ CDM model



In [ ]:
astropy.cosmology.Planck18

Let's put things in a class now. Note **lazy loading**, which is a terrific tecnique!

In [ ]:
class uniformredshift(object):
    def __init__(self,zmax):
        ''' Lazy loading...'''
        self._norm = None
        self._pdfmax = None
        self.zmax=zmax

    def _eval(self,z_vals):
        '''Unnormalized pdf'''
        return ((4.*np.pi*astropy.cosmology.Planck18.differential_comoving_volume(z_vals).value))


    def norm(self):
        '''Compute normalization'''
        if self._norm is None:
            self._norm = scipy.integrate.quad( self._eval, 0, self.zmax)[0]
        return self._norm


    def eval(self,z_vals):
        return self._eval(z_vals)/self.norm()

        return np.array(zsample)

In [ ]:
redshiftpdf = uniformredshift(zmax = 5)

z = np.linspace(0,5,100)
plt.plot(z,2.4*redshiftpdf.eval(z)) ###   Arbitrary normalization, just matching by eye

plt.hist(data,bins=50,histtype='step',density=True,label='original data');


They are not distributed unifiormly in comoving volume! I mean, they are but only at low redshits.

Surely are all quasars created equally? But do we see them equally?